In [144]:
import pandas as pd
import numpy as np
from IPython.display import display
import pickle

In [145]:
movies = pd.read_csv('movie_data_with_urls.csv')
ratings = pd.read_csv('ratings.csv')

In [146]:
movies

,movieId,title,genres,Poster_URL
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,2,Jumanji (1995),Adventure|Children|Fantasy,https://image.tmdb.org/t/p/original/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg
2,3,Grumpier Old Men (1995),Comedy|Romance,https://image.tmdb.org/t/p/original/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,https://image.tmdb.org/t/p/original/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
4,5,Father of the Bride Part II (1995),Comedy,https://image.tmdb.org/t/p/original/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg
...,...,...,...,...
10328,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy,https://image.tmdb.org/t/p/original/vHzbxEcwN17ZGcvHiN8jRZywqZH.jpg
10329,146878,Le Grand Restaurant (1966),Comedy,https://image.tmdb.org/t/p/original/gTr2HfSlRpKf4KtJYyIQwNLADGm.jpg
10330,148238,A Very Murray Christmas (2015),Comedy,https://image.tmdb.org/t/p/original/2ir2DjNNXmMePsyc4inFa8oI23r.jpg
10331,148626,The Big Short (2015),Drama,https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg


### Data Preprocessing

In [147]:
movList = list(movies['genres'])
sp_list = []

for mov in movList:
    sp_list.append(mov.split('|'))

movies['genres'] = sp_list
movies

,movieId,title,genres,Poster_URL
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",https://image.tmdb.org/t/p/original/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",https://image.tmdb.org/t/p/original/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",https://image.tmdb.org/t/p/original/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
4,5,Father of the Bride Part II (1995),[Comedy],https://image.tmdb.org/t/p/original/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg
...,...,...,...,...
10328,146684,Cosmic Scrat-tastrophe (2015),"[Animation, Children, Comedy]",https://image.tmdb.org/t/p/original/vHzbxEcwN17ZGcvHiN8jRZywqZH.jpg
10329,146878,Le Grand Restaurant (1966),[Comedy],https://image.tmdb.org/t/p/original/gTr2HfSlRpKf4KtJYyIQwNLADGm.jpg
10330,148238,A Very Murray Christmas (2015),[Comedy],https://image.tmdb.org/t/p/original/2ir2DjNNXmMePsyc4inFa8oI23r.jpg
10331,148626,The Big Short (2015),[Drama],https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg


In [148]:
movies['genres'] = movies['genres'].apply(lambda genre_list: [genre.lower() for genre in genre_list])
movies['title'] = movies['title'].str.lower()
movies

,movieId,title,genres,Poster_URL
0,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,2,jumanji (1995),"[adventure, children, fantasy]",https://image.tmdb.org/t/p/original/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg
2,3,grumpier old men (1995),"[comedy, romance]",https://image.tmdb.org/t/p/original/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg
3,4,waiting to exhale (1995),"[comedy, drama, romance]",https://image.tmdb.org/t/p/original/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
4,5,father of the bride part ii (1995),[comedy],https://image.tmdb.org/t/p/original/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg
...,...,...,...,...
10328,146684,cosmic scrat-tastrophe (2015),"[animation, children, comedy]",https://image.tmdb.org/t/p/original/vHzbxEcwN17ZGcvHiN8jRZywqZH.jpg
10329,146878,le grand restaurant (1966),[comedy],https://image.tmdb.org/t/p/original/gTr2HfSlRpKf4KtJYyIQwNLADGm.jpg
10330,148238,a very murray christmas (2015),[comedy],https://image.tmdb.org/t/p/original/2ir2DjNNXmMePsyc4inFa8oI23r.jpg
10331,148626,the big short (2015),[drama],https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg


In [149]:
display(movies.head(), ratings.head())

,movieId,title,genres,Poster_URL
0,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,2,jumanji (1995),"[adventure, children, fantasy]",https://image.tmdb.org/t/p/original/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg
2,3,grumpier old men (1995),"[comedy, romance]",https://image.tmdb.org/t/p/original/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg
3,4,waiting to exhale (1995),"[comedy, drama, romance]",https://image.tmdb.org/t/p/original/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
4,5,father of the bride part ii (1995),[comedy],https://image.tmdb.org/t/p/original/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [150]:
mov_ratings = movies.merge(ratings, how='inner', on='movieId')
mov_ratings.drop('timestamp', axis=1, inplace=True)
mov_ratings.head()

,movieId,title,genres,Poster_URL,userId,rating
0,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,2,5.0
1,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,5,4.0
2,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,8,5.0
3,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,11,4.0
4,1,toy story (1995),"[adventure, animation, children, comedy, fantasy]",https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,14,4.0


### Content Based Filtering

In [151]:
mov_ratings[mov_ratings['title'] == 'Cocoon (1985)']

,movieId,title,genres,Poster_URL,userId,rating


In [152]:
new_df = mov_ratings[['movieId', 'title', 'genres', 'Poster_URL']]


In [153]:
# remove square brackets
new_df['genres'] = new_df['genres'].apply(lambda x: ' '.join(x))
new_df.drop_duplicates(inplace=True)

C:\Users\ranuja\AppData\Local\Temp\ipykernel_8772\730877714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['genres'] = new_df['genres'].apply(lambda x: ' '.join(x))
C:\Users\ranuja\AppData\Local\Temp\ipykernel_8772\730877714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(inplace=True)


In [154]:
new_df.shape

(10325, 4)

In [155]:
new_df = new_df.reset_index()
new_df.head()

,index,movieId,title,genres,Poster_URL
0,0,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg
1,232,2,jumanji (1995),adventure children fantasy,https://image.tmdb.org/t/p/original/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg
2,324,3,grumpier old men (1995),comedy romance,https://image.tmdb.org/t/p/original/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg
3,382,4,waiting to exhale (1995),comedy drama romance,https://image.tmdb.org/t/p/original/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg
4,393,5,father of the bride part ii (1995),comedy,https://image.tmdb.org/t/p/original/rj4LBtwQ0uGrpBnCELr716Qo3mw.jpg


In [156]:
# To transform given text into a vector on the basis of frequency count
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=23, stop_words='english')

In [157]:
vectors = cv.fit_transform(new_df['genres']).toarray()
vectors.shape

(10325, 23)

In [158]:
from nltk.stem.porter import PorterStemmer
pt = PorterStemmer()

In [159]:
def stem(text):
    y = []
    for i in text.split():
        y.append(pt.stem(i))
    
    return " ".join(y)

In [160]:
new_df['genres'] = new_df['genres'].apply(stem)
new_df['genres'][0]

'adventur anim children comedi fantasi'

In [161]:
# Save the array to a pickle file
with open('./PKL_Files/stemmed_df_content_based', 'wb') as file:
    pickle.dump(new_df, file)

In [162]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [163]:
# Save the array to a pickle file
with open('./PKL_Files/similarity_content_based', 'wb') as file:
    pickle.dump(similarity, file)

In [164]:
def recommend(movie):
    # load files
    # Load the array from the pickle file
    with open('./PKL_Files/stemmed_df_content_based', 'rb') as file:
        new_df = pickle.load(file)

        # Load the array from the pickle file
    with open('./PKL_Files/similarity_content_based', 'rb') as file:
        similarity = pickle.load(file)

    mov_list = []
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in movie_list:
        d = dict()
        d['title'] = new_df.iloc[i[0]].title
        d['url'] = new_df.iloc[i[0]].Poster_URL
        mov_list.append(d)
    
    return mov_list

In [166]:
recommended_mov_list = recommend('toy story (1995)')
for m in recommended_mov_list:
    print(m)

{'title': 'antz (1998)', 'url': 'https://image.tmdb.org/t/p/original/lWPjxbUMpAHFkJpZHHNWhQaRsax.jpg'}
{'title': 'toy story 2 (1999)', 'url': 'https://image.tmdb.org/t/p/original/2MFIhZAW0CVlEQrFyqwa4U6zqJP.jpg'}
{'title': 'adventures of rocky and bullwinkle, the (2000)', 'url': 'https://image.tmdb.org/t/p/original/xCFSsftt2rglC81I6QLWcZSTCBM.jpg'}
{'title': "emperor's new groove, the (2000)", 'url': 'https://image.tmdb.org/t/p/original/wwbgkXQBEKtnyIJapk6gUgWkVw8.jpg'}
{'title': 'monsters, inc. (2001)', 'url': 'https://image.tmdb.org/t/p/original/sgheSKxZkttIe8ONsf2sWXPgip3.jpg'}


### Colaborative filtering

In [167]:
mov_ratings[mov_ratings['title'] == 'nothing in common (1986)']

,movieId,title,genres,Poster_URL,userId,rating
51756,2418,nothing in common (1986),[comedy],https://image.tmdb.org/t/p/original/l9XCfmB2aYWpkio8ltuGDTm8s1S.jpg,560,3.5
51757,2418,nothing in common (1986),[comedy],https://image.tmdb.org/t/p/original/l9XCfmB2aYWpkio8ltuGDTm8s1S.jpg,575,3.0


In [168]:
# remove square brackets
pd.set_option('display.max_colwidth', None)

mov_ratings['genres'] = mov_ratings['genres'].apply(lambda x: ' '.join(x))
mov_ratings.drop_duplicates(inplace=True)


In [169]:
# Save the array to a pickle file
with open('./PKL_Files/movie_rating_collaborative', 'wb') as file:
    pickle.dump(mov_ratings, file)

In [170]:
# users who have given more than 100 ratings are considered

x = mov_ratings.groupby('userId').count()['rating'] > 100
users = x[x].index
users

Index([  1,   4,   7,   9,  17,  22,  24,  25,  28,  29,
       ...
       637, 642, 650, 655, 656, 659, 662, 665, 666, 668],
      dtype='int64', name='userId', length=253)

In [171]:
filtered_rating = mov_ratings[mov_ratings['userId'].isin(users)]
filtered_rating


,movieId,title,genres,Poster_URL,userId,rating
5,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,17,5.0
6,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,28,3.0
7,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,29,4.0
10,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,32,4.0
11,1,toy story (1995),adventure animation children comedy fantasy,https://image.tmdb.org/t/p/original/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,38,5.0
...,...,...,...,...,...,...
105375,148238,a very murray christmas (2015),comedy,https://image.tmdb.org/t/p/original/2ir2DjNNXmMePsyc4inFa8oI23r.jpg,475,3.0
105376,148626,the big short (2015),drama,https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg,458,4.0
105377,148626,the big short (2015),drama,https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg,576,4.5
105378,148626,the big short (2015),drama,https://image.tmdb.org/t/p/original/isuQWbJPbjybBEWdcCaBUPmU0XO.jpg,668,4.5


In [172]:
# movies with more than 50 ratings
y = filtered_rating.groupby('title').count() > 50
famous_movies = y[y].index
famous_movies

Index([''71 (2014)', ''burbs, the (1989)', ''night mother (1986)',
       ''round midnight (1986)', ''til there was you (1997)',
       '(500) days of summer (2009)', '*batteries not included (1987)',
       '...and justice for all (1979)', '10 (1979)', '10 items or less (2006)',
       ...
       'zone 39 (1997)', 'zookeeper (2011)', 'zoolander (2001)',
       'zorba the greek (alexis zorbas) (1964)', 'zorro, the gay blade (1981)',
       'zozo (2005)',
       'zu: warriors from the magic mountain (xin shu shan jian ke) (1983)',
       'zulu (1964)', '¡three amigos! (1986)',
       'à nous la liberté (freedom for us) (1931)'],
      dtype='object', name='title', length=10202)

In [173]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_movies)]



In [174]:
pt = final_ratings.pivot_table(index='title',columns='userId',values='rating')
pt.fillna(0,inplace=True)
pt

userId,1,4,7,9,17,22,24,25,28,29,...,637,642,650,655,656,659,662,665,666,668
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, the (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'night mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'round midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
'til there was you (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zozo (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
zu: warriors from the magic mountain (xin shu shan jian ke) (1983),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zulu (1964),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [175]:
# Save the array to a pickle file
with open('./PKL_Files/pivot_table_collaborative', 'wb') as file:
    pickle.dump(pt, file)

In [176]:
similarity_scores = cosine_similarity(pt)
similarity_scores

array([[1.        , 0.        , 0.        , ..., 0.        , 0.10919022,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.20265471,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.40492914, 0.10919022,
        0.        ],
       ...,
       [0.        , 0.        , 0.40492914, ..., 1.        , 0.19159531,
        0.        ],
       [0.10919022, 0.20265471, 0.10919022, ..., 0.19159531, 1.        ,
        0.10919022],
       [0.        , 0.        , 0.        , ..., 0.        , 0.10919022,
        1.        ]])

In [177]:
# Save the array to a pickle file
with open('./PKL_Files/similarity_scores_collaborative', 'wb') as file:
    pickle.dump(similarity_scores, file)

In [178]:
def collaborative_recommend(movie_name):
    # Load files
    with open('./PKL_Files/movie_rating_collaborative', 'rb') as file:
        mov_ratings = pickle.load(file)

    with open('./PKL_Files/similarity_scores_collaborative', 'rb') as file:
        similarity_scores = pickle.load(file)

    with open('./PKL_Files/pivot_table_collaborative', 'rb') as file:
        pt = pickle.load(file)

    # index fetch
    index = np.where(pt.index==movie_name)[0][0]

    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    # simillar items from 1 to 4
    
    data = []
    for i in similar_items:
        item = []
        d = dict()
        temp_df = mov_ratings[mov_ratings['title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))

        # urls = temp_df['Poster_URL'].str.replace(r'\d+', '', regex=True) # to remove the index value before the RL

        d['title'] = item[0]
        d['url'] = temp_df['Poster_URL'].values.tolist()[0]
        data.append(d)
    
    return data

In [179]:
collaborative_recommend('zulu (1964)')

[{'title': 'nothing in common (1986)',
  'url': 'https://image.tmdb.org/t/p/original/l9XCfmB2aYWpkio8ltuGDTm8s1S.jpg'},
 {'title': 'undercover blues (1993)',
  'url': 'https://image.tmdb.org/t/p/original/r9yKccGLLzpPwRCyfCMjG2oNcbF.jpg'},
 {'title': 'zorro, the gay blade (1981)',
  'url': 'https://image.tmdb.org/t/p/original/qNFtu1TyoFbmG1hCdQkD8ENOjum.jpg'},
 {'title': 'black book (zwartboek) (2006)',
  'url': 'https://image.tmdb.org/t/p/original/kn28W24slBLyGr8ZIZnxNE5YZrY.jpg'}]